## The common experiment functions
### The functions are used to perform model optimization & hyper parameters tunning
### After the tunning, the best parameters a combined to come up with the best model

In [28]:
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, MaxPooling2D, BatchNormalization, Conv2D, Flatten
from keras.models import Sequential
from keras import optimizers
from sklearn.model_selection import train_test_split

In [29]:
#---------------------------------------------------------------------
# Function to create a default configuration for our neural network model
# The configuration can be overridden while creating the model
#
def base_model_config():
    model_config = {
        "HIDDEN_NODES" : [32,64],
        "HIDDEN_ACTIVATION" : "relu",
        "OUTPUT_NODES" : 3,
        "NEAR_OUTPUT_NODES": 512,
        "OUTPUT_ACTIVATION" : "sigmoid",
        "WEIGHTS_INITIALIZER" : "glorot_uniform",
        "BIAS_INITIALIZER" : "zeros",
        "NORMALIZATION" : "none",
        "OPTIMIZER" : "adam",
        "LEARNING_RATE" : 0.001,
        "REGULARIZER" : None,
        "DROPOUT_RATE" : 0.0,
        "DROPOUT_RATE_LAST_LAYER": 0.5,
        "EPOCHS" : 10,
        "BATCH_SIZE" : 16,
        "VALIDATION_SPLIT" : 0.2,
        "VERBOSE" : 0,
        "LOSS_FUNCTION" : "binary_crossentropy",
        "METRICS" : ["accuracy"],
        "PADDING": "same"
    }
    return model_config

In [30]:
#---------------------------------------------------------------------
# Function to create an optimizer based on the optimizer name and learning rate
#---------------------------------------------------------------------
def get_optimizer(optimizer_name, learning_rate):
    # adagrad, adam, rmsprop, and sgd
    optimizer = None

    if optimizer_name == "adagrad":
        optimizer = optimizers.Adagrad(learning_rate=learning_rate)
    elif optimizer_name == "adam":
        optimizer = optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "rmsprop":
        optimizer = optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = optimizers.SGD(learning_rate=learning_rate)

    return optimizer

In [31]:
#---------------------------------------------------------------------
# Function to create and fit the deep neural network model
#
def create_and_run_model(model_config, X, Y, model_name):
    model = Sequential(name = model_name)

    for layer in range(len(model_config["HIDDEN_NODES"])):
        if layer == 0:
            model.add(
                Conv2D(
                    (),
                    padding= model_config["PADDING"],
                    activation = model_config["HIDDEN_ACTIVATION"],
                )
            )
        else:
            if model_config["NORMALIZATION"] == "batch":
                model.add(BatchNormalization())
            if model_config["DROPOUT_RATE"] > 0.0:
                model.add(Dropout(model_config["DROPOUT_RATE"]))

            model.add(
                Conv2D(
                    (),
                    padding= model_config["PADDING"],
                    activation = model_config["HIDDEN_ACTIVATION"],
                    # model_config["HIDDEN_NODES"][layer],
                    bias_initializer = model_config["BIAS_INITIALIZER"],
                    kernel_regularizer = model_config["REGULARIZER"],
                    kernel_initializer = model_config["WEIGHTS_INITIALIZER"],
                    name="Dense-Layer-" + str(layer),
                ),
            )
            
    # Flatten the Conv2D layer output
    model.add(Flatten())

    # Add Dense near output layer to the model
    model.add(
        Dense(
            model_config["NEAR_OUTPUT_NODES"],
            name="Near-Output-Layer",
            activation=model_config["OUTPUT_ACTIVATION"]
        )
    )

    # Dropout 50 % of the neural network connection
    model.add(Dropout(model_config["DROPOUT_RATE_LAST_LAYER"]))

    # Add ouput layer to the model
    model.add(
        Dense(
            model_config["OUTPUT_NODES"],
            name="Output-Layer",
            activation=model_config["OUTPUT_ACTIVATION"]
        )
    )

    # Get optimizer
    optimizer = get_optimizer(model_config["OPTIMIZER"], model_config["LEARNING_RATE"])

    # Compile the model
    model.compile(
        loss = model_config["LOSS_FUNCTION"],
        optimizer=optimizer,
        metrics=model_config["METRICS"]
    )

    # Print model Summary
    model.summary()

    # Split the data into training set and validation set
    X_train, x_val, Y_train, y_val = train_test_split(
        X,
        Y,
        stratify= Y,
        test_size=model_config["VALIDATION_SPLIT"]
    )

    # Fit the model
    history = model.fit(X_train,
                      Y_train,
                      batch_size=model_config["BATCH_SIZE"],
                      epochs=model_config["EPOCHS"],
                      verbose=model_config["VERBOSE"],
                      validation_data= (x_val, y_val))

    return history


In [32]:
#---------------------------------------------------------------------
# Function to plot a graph based on the results of the model
#---------------------------------------------------------------------

def plot_graph(accuracy_measures, title):

    plt.figure(figsize=(15, 8))

    for experiment in accuracy_measures.keys():
        plt.plot(
            accuracy_measures[experiment],
            label=experiment,
            linewidth=3
        )

    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()